# RSA

## Symmetrische Verschlüsselung
Die Verschlüsselung und die Entschlüsselung werden mit dem identischen Schlüssel durchgeführt.
Beispiel: Caesar-Schiffre --> Der Schlüssel ist beispielsweise 2 --> das bedeutet, dass alle Buchstaben im Alphabet um 2 Stellen verschoben werden.

## Asymmetrische Verschlüsselung
Es existiert ein private und ein public Key.
Beispiel: Wenn Alice eine Nachricht an Bob senden will, verschlüsselt sie ihre Nachricht mit dem public key von Bob und sendet die Nachricht. Jetzt kann nur Bob die Nachricht entschlüsseln, da nur er den private key hat.

In der Praxis werden die Verfahren kombiniert. Die eigentliche Übertragung der Nachrichten erfolgt symmetrisch. Für die Übertragung des Schlüssel wird ein Asymmetrisches Verfahren verwendet. Also wird für jede Sitzung ein Sitzungsschlüssel asymmetrisch übertragen, der dann für die symmetrische Verschlüsselung verwendet wird.

## RSA Infos
RSA ist ein asymmetrisches Verschlüsselungsverfahren, das auf der Schwierigkeit beruht, grosse Primzahlen in ihre Faktoren zu zerlegen. Es wurde 1977 von Riverst, Shamir und Adleman am MIT entwickelt.

## So wird ein Schüsselpaar erzeugt
- Wähle zwei Primzahlen p und q aus
	- In der Praxis sind dies Zahlen mit ca. 1024 Bits (ca. 600 Dezimalstellen) und werden mit Primzahltests gesucht
- Bilde das Produkt n = pq
- Berechne phi von n = $\phi$(n) = (p-1)(q-1)  |  Also einfach p-1 mal q-1
- Ermittle eine Zahl e, die teilerfremd zu $\phi$(n) ist
- Berechne den Kehrwert d von e modulo $\phi$(n)
	- Das würde man mit dem euklidischen Algorithmus machen --> oder einfach e<sup>-1</sup> mod $\phi$(n)
- Fertig, wir haben e, d und n
	- e (encrypt) und n bilden den public-key
	- d (decrypt) und n bilden den private-key

Das ganze wird nur ein Mal gemacht und es können e, d und n immer wieder verwendet werden. Solange d geheim bleibt...

## So wird verschlüsselt und entschlüsselt
Nachrichten sind positive ganze Zahlen, die kleiner als n sind.
Eine Nachricht wird mit dem öffentlichen Schlüssel verschlüsselt.
S = N<sup>e</sup> mod n
Diese Berechnung würde in der Praxis mit sehr grossen Zahlen durchgeführt werden und muss daher sehr effizient gelöst werden. Ein binäre Exploitation kombiniert mit der modularen Arithmetik ist dabei essenziell.

Beispiel:

Wir wollen die Nachricht 52134 versenden.

N = 52134

S = 52134<sup>e</sup> mod n

S ist die verschlüsselte zu übermittelnde Nachricht.

Das Entschlüsseln funktioniert dann mit dem privaten Schlüssel:

N = S<sup>d</sup> mod n

### Wieso klappt das? Wieso gilt modulo n immer (N<sup>e</sup>)<sup>d</sup> = N?
Kurz gesagt:
Wenn N kein vielfaches von p ist, dann kann der kleine Satz von Fermat angewendet werden.
--> N<sup>p-1</sup> = 1
--> N<sup>ed</sup> = N

Wenn N ein vielfaches von p ist, also N = 0 in Z<sub>p</sub> und damit natürlich auch N<sup>ed</sup> = N

Wir haben somit ***p | N<sup>ed</sup> - N***  --> p ist ein Teiler der Differenz von N<sup>ed</sup> - N
Da p und q teilerfremd sind, folgt ***pq | N<sup>ed</sup> - N*** --> pq ist ein Teiler der Differenz von N<sup>ed</sup> - N

### Wie kann man den Code knacken?
- Kennt man p und q, so kann man sofort $\phi$(n) = (p-1)(q-1) berechnen
- Da e öffentlich ist, kann man dann auch d ermitteln
	- weil d der Kehrwert von e und phi ist --> d = e modulo $\phi$(n)
- Und so kommt man an den private key

Die Sicherheit des Verfahrens beruht also auf zwei "Erfahrungswerten":
- Ist n gross genug, so ist es schwer, n in Primfaktoren zu zerlegen
	- Es ist kein effizientes Verfahren bekannt, womit man eine grosse Zahl in seine Primfaktoren zerlegen kann
- Modulo n ist es schwer, N zu bestimmen, wenn man N<sup>e</sup> und e kennt
	- Auch hier gibt es kein bekanntes effizientes Verfahren

Es gibt aber keinen mathematischen Beweis dafür, dass dies ein sicheres Verschlüsselungsverfahren ist. Mit Quantencomputern kann das Verfahren geknackt werden, da es bereits einen Algorithmus gibt, der grosse Zahlen im seine Primfaktoren zerlegen kann.

## Weitere Anwendungsmöglichkeit
Alice könnte mit ihrem private key etwas verschlüsseln und an Bob senden. Bob kann dies mit dem public key von Alice entschlüsseln und mit der unverschlüsselten Version vergleichen und somit überprüfen, ob die Nachricht tatsächlich von Alice stammt.

In [25]:
# Beispiel RSA Schlüsselpaar erzeugen

p = 101
q = 127
n = p * q
phi = (p - 1) * (q - 1)
e = 11  # e muss teilerfremd zu phi sein
# das kleinste e kann einfach ermittelt werden --> phi / 2 rechnen, dann /3, etc. Bis das erste Mal eine Kommazahl rauskommt, dies ist dann eine teilerfremde Zahl
d = pow(e, -1, phi)  # d ist der Kehrwert von e
print(f'p: {p}, q: {q}, n: {n}, phi: {phi}, e: {e}, d: {d}')
print(f'Public Key: (e: {e}, n: {n})')
print(f'Private Key: (d: {d}, n: {n})')

p: 101, q: 127, n: 12827, phi: 12600, e: 11, d: 2291
Public Key: (e: 11, n: 12827)
Private Key: (d: 2291, n: 12827)


In [26]:
# Beispiel RSA Verschlüsselung und Entschlüsselung

N = 57
S = pow(N, e, n)  # Verschlüsselung

print(f'Original Nachricht: {N}')
print(f'Verschlüsselte Nachricht: {S}')

# Und hier entschlüsseln wir eine Nachricht
S = 69
N = pow(S, d, n)  # Entschlüsselung

print(f'Verschlüsselte Nachricht: {S}')
print(f'Original Nachricht: {N}')


Original Nachricht: 57
Verschlüsselte Nachricht: 63
Verschlüsselte Nachricht: 69
Original Nachricht: 8920


# Das ElGamal-Verfahren
Hier wird die Idee von Diffi-Hellman zur Verschlüsselung von Nachrichten verwendet.

1. A wählt eine Primzahl p und eine Primitivwurzel g modulo p
	1. 2 $\le$  g $\le$ p - 2. Es ist also g<sup>p-1</sup> = 1 (p)
2. Dann wählt A zufällig x $\in$ {1, 2, ..., p-2} und berechnet X = g<sup>x</sup> (p)
	1. Das Tripel (p, g, X) bildet A's öffentlicher Schlüssel und x ist ihr geheimer Schlüssel
3. Damit B eine Nachricht verschlüsselt an A senden kann, nimmt sie den öffentlichen Schüssel von A, wählt zufällig y $\in$ {1,2,..., p-2} und berechnet Y = g<sup>y</sup> (p)
4. B verschlüsselt den Klartext m $\in$ {0, 1, ..., p-1} zum Ciphertext c = X<sup>y</sup> * m (p) und sendet das Paar (Y, c) an A
5. A berechnet Y<sup>d</sup> * c (p) mit d = p -1 - x und erhält damit den Klartext m

Beispiel:
1. A wählt p = 7 und g = 3
2. A wählt x = 2 und berechnet X = 3<sup>2</sup> (p) = 2
	1. Der öffentliche Schlüssel von A ist das Tripel (7, 3, 2)
3. B wählt y = 4 und berechnet Y = 3<sup>4</sup> (p) = 4
4. B verschlüsselt den Klartext m = 6 zum Ciphertext c = 2<sup>4</sup> * m (p) = 16 * 6 (p) = 96 (p) = 5 und sendet das Paar (4, 5) an A
5. A berechnet d = 7 - 1 - 2 = 4 und
   4<sup>4</sup> * 5 (p) = 256 * 5 (p) = 1280 (p) = 6
6. Der Klartext ist also die 6!

In [11]:
def find_lowest_generator(prim: int) -> int:
    """
    Find the lowest generator for Z_p.
    :param prim: Prime.
    :returns: The lowest generator for Z_p.
    """
    for generator in range(2, prim):
        if len({pow(generator, i, prim) for i in range(prim)}) == prim - 1:
            return generator
    return 0

In [27]:
#print Beispiel ElGamal privater Schlüssel vorhanden und jetzt öffentlichen bestimmen
p = 23
g = find_lowest_generator(p)
x = 10

X = pow(g, x, p)

print(f'Public Key: (p: {p}, g: {g}, X: {X})')
print(f'Private Key: (p: {p}, g: {g}, x: {x})')

Public Key: (p: 127, g: 3, X: 77)
Private Key: (p: 127, g: 3, x: 10)


In [29]:
# Beispiel ElGamal Nachricht mit public key verschlüsseln
m = 17
y = 15

Y = pow(g, y, p)
c = (X ** y) * m % p

print(f'Ciphertext: (Y: {Y}, c: {c})')

Ciphertext: (Y: 116, c: 115)


In [15]:
# Beispiel ElGamal Nachricht mit private key entschlüsseln
Y = 19
c = 10
print(f'Ciphertext: (Y: {Y}, c: {c})')

d = p - 1 - x
m = (Y ** d) * c % p  # Auch möglich ist: (pow(Y, d, p) * c) % p

print(f'Klartext: {m}')

Ciphertext: (Y: 19, c: 10)
Klartext: 17


# Unterschied RSA und ElGamal
- Bei RSA ist das Chiffrat gleich gross wie der Klartext, bei ElGamal ist das Chiffrat doppelt so gross wie der Klartext
- Falls ein schneller Algorithmus für die Primfaktorzerlegung gefunden würde, wäre RSA gebrochen
- ElGamal ist sicher, solange das diskrete Logarithmusproblem schwer zu lösen ist
- ElGamal ist probabilistisch, d.h. die Verschlüsselung derselben Nachricht liefert jedes Mal ein anderes Chiffrat
- ElGamal ist etwas langsamer als RSA
- ElGamal wird in der Praxis häufiger verwendet als RSA
- Beide Varianten können auch für digitale Signaturen verwendet werden
- Quantencomputern könnten sowohl das Faktorisierungsproblem als auch das diskrete Logarithmusproblem effizient lösen, was beide Verfahren unsicher machen würde --> Bis heute können noch keine ausreichend grosse Quantencomputer dazu gebaut werden

## Vergleich der Schlüssellängen gemäss NIST
| Symmetrisch (in Bit) | RSA (in Bit) | ElGamal (in Bit) |
|----------------------| ------------ |------------------|
| 80                   | 1024         | 160              |
| 112                  | 2048         | 224              |
| 128                  | 3072         | 256              |
| 192                  | 7680         | 384              |
| 256                  | 15360        | 521              |

